In [2]:
import azureml.core 
from azureml.core import Workspace


In [3]:
ws = Workspace.from_config() #get default workspace

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig,Model
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
#from azureml.core.webservice import LocalWebservice
#deployment_config = LocalWebservice.deploy_configuration(port=8890)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service1"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.get_logs())

In [4]:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-service1
diabetes-service


### single row predictions

In [7]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = ws.webservices['diabetes-service1'].run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
not-diabetic


### multiple row predictions

In [8]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = ws.webservices['diabetes-service1'].run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] not-diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] diabetic


URL to which these applications must submit their requests

In [9]:
endpoint = ws.webservices['diabetes-service1'].scoring_uri #service.scoring_uri
print(endpoint)

http://1687af0a-9f21-47ce-bfdc-9bf8967794e0.centralindia.azurecontainer.io/score


### use above uri to call the prediction service

In [10]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] not-diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] diabetic


### delete service

In [11]:
ws.webservices['diabetes-service1'].delete()
print ('Service deleted.')

ws.webservices['diabetes-service'].delete()
print ('Service deleted.')

Service deleted.


KeyError: 'diabetes-service'